In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

In [2]:
import time
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = get_human_dataset()

start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)

pred = gb_clf.predict(X_test)
accuracy = accuracy_score(pred, y_test)

print("GBM 정확도: {0:.4f}".format(accuracy))
print("GBM 수행시간: {0:.1f} 초".format(time.time()-start_time))

GBM 정확도: 0.9382
GBM 수행시간: 665.5 초


In [ ]:
'''
GBM의 하이퍼 파라미터는 트리 기반 파라미터 외에 몇가지가 더 있습니다.
loss는 경사 하강법에서 사용할 비용 함수를 지정합니다. 기본값은 deviance입니다.
learning_rate는 학습 진행률입니다. 기본값은 0.1입니다.
n_estimators는 weak learner의 개수입니다. 개수가 많을수록 성능이 좋아지지만 그만큼 수행시간이 길어집니다. 기본값은 100입니다.
subsample는 weak learner가 학습에 사용하는 데이터의 샘플링 비율입니다. 
'''

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': [100,500],
         'learning_rate': [0.05,0.1]}

grid_cv = GridSearchCV(gb_clf, param_grid = params, cv = 2, verbose = 1)
grid_cv.fit(X_train, y_train)
print("최적 하이퍼 파라미터: \n", grid_cv.best_params_ )
print("accuracy: {0:.4f}".format(grid_cv.best_score_))

In [ ]:
##실제 수행시간이 30분 이상 걸려서 돌려보면 다음과 같습니다.
{learning_rateL 0.05, n_estimators: 500}
accuracy: 0.9010

## GBM은 과적합에도 강한 뛰어난 예측 성능을 보여주지만 수행 시간이 오래 걸린다는 단점이 있습니다. 
## 수행시간을 보다 빠르게한 XGBoost와 LightGBM에 대해서 다음시간에 다뤄보도록 하겠습니다.